# Import packages and data 

In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
import scipy.stats
import anndata
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.axes._axes import _log as matplotlib_axes_logger
from scipy import sparse
matplotlib_axes_logger.setLevel('ERROR')
# silence NumbaPerformanceWarning
import warnings
from numba.errors import NumbaPerformanceWarning
warnings.filterwarnings("ignore", category=NumbaPerformanceWarning)

/Users/b8058304/.pyenv/versions/3.6.4/envs/py36/lib/python3.6/site-packages/anndata/_core/anndata.py:21: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex


In [2]:
sc.settings.verbosity = 1  # verbosity: errors (0), warnings (1), info (2), hints (3)
# Set up the plot config for viewing the annotation clearly.
sc.settings.set_figure_params(dpi=120, dpi_save=1000)
sc.logging.print_versions()

scanpy==1.4.4 anndata==0.7.1 umap==0.3.10 numpy==1.17.1 scipy==1.4.1 pandas==1.0.5 scikit-learn==0.22.2.post1 statsmodels==0.11.1 python-igraph==0.8.0 louvain==0.6.1


# Import the fetal YS myeloid

In [3]:
adata = sc.read('/Users/b8058304/Documents/PhD_work/Coding/liver/data/ys_total_annotation_20191127.h5ad')

In [4]:
ys = adata

In [5]:
adata

AnnData object with n_obs × n_vars = 10071 × 32084 
    obs: 'nGene', 'nUMI', 'orig.ident', 'percent.mito', 'cell.labels', 'fetal.ids', 'sort.ids', 'tissue', 'lanes', 'stages', 'sample.type', 'gender', 'AnnatomicalPart', 'doublets', 'LouvainClustering', 'predicted.cell.labels', 'full.ids', 'batch', 'bh_pval', 'sample', 'scrublet_cluster_score', 'scrublet_score', 'n_counts', 'louvain', 'is_doublet', 'is_doublet_poptrim', 'res.30', 'backlabeled_1', 'cell.labels_progen'

In [6]:
cell_numbers = adata.obs.groupby(["cell.labels"]).apply(len)
cell_numbers

cell.labels
DC progenitor                78
Early mast cell             133
Endothelium                 160
Erythroid AB                892
Erythroid EZ               1167
Erythroid M                 275
ILC precursor                25
Lymphoid progenitor         184
MEMP                        121
Monocyte                     38
Myeloid progenitor           61
NK early                     50
yolk sac Macrophage I      3063
yolk sac Macrophage II     3619
yolk sac progenitor/MPP     205
dtype: int64

In [7]:
cell_numbers = adata.obs.groupby(["cell.labels_progen"]).apply(len)
cell_numbers

cell.labels_progen
CMP                    55
ELP                   174
GMP                    40
HSC                   105
MEMP                   55
MEP                    56
Monocyte precursor     26
macrophage             42
neut myeloid           18
dtype: int64

In [8]:
from scipy import sparse
array_vals = adata.X
adata.X = sparse.csr_matrix(adata.X)
adata.raw = adata
adata.X = array_vals

In [9]:
# GMP, monocyte precursor, neut myeloid (from the original  Lymphoid progenitor, MEMP, Myeloid progenitor and 
# yolk sac progenitor/MPP reclustering on right hand side of slide one) as well as monocyte and DC progenitor from 
# the original annotation
adata_1 = adata[adata.obs['cell.labels'].isin(['DC progenitor', 'Monocyte'])].copy()
adata_2 = adata[adata.obs['cell.labels_progen'].isin(['GMP', 'Monocyte precursor', 'neut myeloid'])].copy()
adata_2.obs['cell.labels'] = adata_2.obs['cell.labels_progen']

In [10]:
cell_numbers = adata_1.obs.groupby(["cell.labels"]).apply(len)
cell_numbers

cell.labels
DC progenitor    78
Monocyte         38
dtype: int64

In [11]:
cell_numbers = adata_2.obs.groupby(["cell.labels"]).apply(len)
cell_numbers

cell.labels
GMP                   40
Monocyte precursor    26
neut myeloid          18
dtype: int64

In [12]:
cell_numbers = adata_1.obs.groupby(["fetal.ids", "cell.labels"]).apply(len)
pd.DataFrame(cell_numbers).to_csv("/Users/b8058304/Documents/PhD_work/Coding/manuscript_figs/resources_for_pipelines/ys_myeloid_nrs_orig_by_sample_20200724.csv")
cell_numbers

fetal.ids          cell.labels  
F32_female_7+6PCW  DC progenitor     1
                   Monocyte          2
F35_female_7+6PCW  DC progenitor    13
                   Monocyte          7
F37_female_4PCW    DC progenitor    64
                   Monocyte         29
dtype: int64

In [13]:
cell_numbers = adata_2.obs.groupby(["fetal.ids", "cell.labels"]).apply(len)
pd.DataFrame(cell_numbers).to_csv("/Users/b8058304/Documents/PhD_work/Coding/manuscript_figs/resources_for_pipelines/ys_myeloid_nrs_reclustered_by_sample_20200724.csv")
cell_numbers

fetal.ids          cell.labels       
F32_female_7+6PCW  GMP                    1
F35_female_7+6PCW  GMP                    2
                   Monocyte precursor     2
                   neut myeloid           6
F37_female_4PCW    GMP                   37
                   Monocyte precursor    24
                   neut myeloid          12
dtype: int64

# Import the fetal liver myeloid compartment

In [14]:
adata = sc.read('/Users/b8058304/Documents/PhD_work/Coding/liver/data/fetal_liver_alladata_Copy1.h5ad')

In [15]:
liver = adata

In [16]:
adata

AnnData object with n_obs × n_vars = 113063 × 27080 
    obs: 'nGene', 'nUMI', 'orig.ident', 'percent.mito', 'fetal.ids', 'sort.ids', 'tissue', 'lanes', 'stages', 'sample.type', 'gender', 'AnnatomicalPart', 'doublets', 'cell.labels', 'combined.labels', 'batch', 'sample', 'n_counts', 'cell.labels_progen'
    obsm: 'X_fdg', 'X_tsne', 'X_umap'

In [17]:
cell_numbers = adata.obs.groupby(["cell.labels"]).apply(len)
cell_numbers

cell.labels
B cell                            1079
DC1                                336
DC2                               3954
DC precursor                       330
Early Erythroid                  11985
Early lymphoid_T lymphocyte        767
Endothelial cell                  3348
Fibroblast                        1713
HSC_MPP                           3439
Hepatocyte                        2479
ILC precursor                     1726
Kupffer Cell                     24841
Late Erythroid                    3180
MEMP                              1342
Mast cell                         1308
Megakaryocyte                     3983
Mid Erythroid                    27000
Mono-Mac                          6590
Monocyte                          2586
Monocyte precursor                 350
NK                                6706
Neutrophil-myeloid progenitor      658
Pre pro B cell                     234
VCAM1+ EI macrophage               161
pDC precursor                      253
pre-B cell   

In [18]:
from scipy import sparse
array_vals = adata.X
adata.X = sparse.csr_matrix(adata.X)
adata.raw = adata
adata.X = array_vals

In [19]:
cell_numbers = adata.obs.groupby(["cell.labels_progen"]).apply(len)
cell_numbers

cell.labels_progen
ELP                        525
GMP                        362
HSC                        242
MEMP                       505
MEP                        639
MPP                       2012
early MK                    81
early erythroid            333
eo/baso/mast precursor      86
erythroid-MPP hybrid        14
macrophage                  81
myeloid DC progenitor       94
pDC progenitor              41
dtype: int64

In [20]:
adata = adata[adata.obs['cell.labels'].isin(['pDC precursor', 'DC precursor', 'DC1', 'DC2', 'Neutrophil-myeloid progenitor', 'Monocyte precursor', 'Monocyte'])].copy()

In [21]:
cell_numbers = adata.obs.groupby(["cell.labels"]).apply(len)
cell_numbers

cell.labels
DC1                               336
DC2                              3954
DC precursor                      330
Monocyte                         2586
Monocyte precursor                350
Neutrophil-myeloid progenitor     658
pDC precursor                     253
dtype: int64

In [22]:
adata.obs["dataset"] = "liver"
adata.obs['cell.labels'] = adata.obs['cell.labels']

In [23]:
cell_numbers = adata.obs.groupby(["fetal.ids", "cell.labels"]).apply(len)
pd.DataFrame(cell_numbers).to_csv("/Users/b8058304/Documents/PhD_work/Coding/manuscript_figs/resources_for_pipelines/liver_myeloid_nrs_by_sample_20200724.csv")
cell_numbers

fetal.ids           cell.labels                  
F16_male_8+1PCW     DC1                                3
                    DC2                               10
                    DC precursor                       1
                    Monocyte                          18
                    Neutrophil-myeloid progenitor      6
                                                    ... 
F45_female_13+6PCW  DC precursor                      37
                    Monocyte                         197
                    Monocyte precursor                73
                    Neutrophil-myeloid progenitor    116
                    pDC precursor                     17
Length: 96, dtype: int64

# Import the FBM myeloid compartment

In [24]:
adata = sc.read('/Users/b8058304/Documents/PhD_work/Coding/bm_plus_19pcw/data/bm_plus_19pcw_raw_dr_20200718.h5ad')

In [25]:
fbm = adata

In [26]:
adata

AnnData object with n_obs × n_vars = 104562 × 33712 
    obs: 'cell.labels', 'doublets', 'fetal.ids', 'gender', 'is_doublet', 'is_doublet_poptrim', 'is_doublet_wolock', 'lanes', 'nGene', 'nUMI', 'orig.ident', 'percent.mito', 'processing.type', 'scrublet_cluster_score', 'scrublet_score', 'sequencing.type', 'sort.ids', 'april_cell.labels', 'cell.labels_20200708', 'cell.labels_20200713', 'cell.labels_20200718', 'nk_meta'
    var: 'gene_ids-1', 'feature_types-1'
    obsm: 'X_orig_pca', 'X_pca', 'X_umap'

In [27]:
cell_numbers = adata.obs.groupby(["cell.labels"]).apply(len)
cell_numbers

cell.labels
CD4 T cell             327
CD8 T cell             171
CD14 monocyte         8787
CD56 bright NK         450
CMP                    425
                      ... 
schwann cells            9
sinusoidal EC          550
stromal macrophage    1493
tDC                    193
tip EC                 363
Length: 64, dtype: int64

In [28]:
from scipy import sparse
array_vals = adata.X
adata.X = sparse.csr_matrix(adata.X)
adata.raw = adata
adata.X = array_vals

In [29]:
adata = adata[adata.obs['cell.labels'].isin(["CD14 monocyte", "DC precursor", "DC1", "DC2", "DC3", 
                     "GMP", "promonocyte", 
                     "myelocyte", "myeloid DC progenitor", "MOP", "neutrophil", "pDC", "pDC progenitor", 
                     "promyelocyte", "tDC"])].copy()

In [30]:
cell_numbers = adata.obs.groupby(["cell.labels"]).apply(len)
cell_numbers

cell.labels
CD14 monocyte            8787
DC1                        50
DC2                       598
DC3                       705
DC precursor              201
GMP                      1285
MOP                      3990
myelocyte                3854
myeloid DC progenitor      31
neutrophil               4516
pDC                       713
pDC progenitor             23
promonocyte              7676
promyelocyte             2386
tDC                       193
dtype: int64

In [31]:
adata.obs["dataset"] = "FBM"

In [32]:
cell_numbers = adata.obs.groupby(["fetal.ids", "cell.labels"]).apply(len)
pd.DataFrame(cell_numbers).to_csv("/Users/b8058304/Documents/PhD_work/Coding/manuscript_figs/resources_for_pipelines/fbm_myeloid_nrs_by_sample_20200724.csv")
cell_numbers

fetal.ids         cell.labels  
F21_male_16+2PCW  CD14 monocyte    217
                  DC1                3
                  DC2               24
                  DC3               33
                  DC precursor      13
                                  ... 
SB19PCW           myelocyte        904
                  neutrophil       767
                  pDC               68
                  promonocyte      798
                  promyelocyte     667
Length: 127, dtype: int64

# Import the ABM myeloid compartment

In [33]:
adata = sc.read('/Users/b8058304/Documents/PhD_work/Coding/adult_bm/data/abm_raw_dr_20200717.h5ad')

In [34]:
abm = adata

In [35]:
adata

AnnData object with n_obs × n_vars = 142026 × 33694 
    obs: 'sample', 'lanes', 'sex', 'age', 'ethnicity', 'mad_prd', 'auto_prd', 'cell.labels', 'cell.labels2', 'leiden', 'percent.mito', 'nGene', 'nUMI', 'cell.labels_july', 'cell.labels_20200717'
    var: 'gene_ids'
    obsm: 'X_orig_pca', 'X_pca', 'X_umap'

In [36]:
cell_numbers = adata.obs.groupby(["cell.labels"]).apply(len)
cell_numbers

cell.labels
CD14 monocyte             3670
CD16 monocyte             1938
CD56 bright NK            1228
CLP                        882
CMP                        288
DC1                        135
DC2                        481
DC3                        550
DC precursor               462
HSC                        497
LMPP                        80
MEMP                       785
MK                         577
MOP                       1440
MPP                        365
Treg                      6327
early MK                   136
early erythroid           5441
erythroid macrophage        77
immature B cell           2728
late erythroid            1150
mature CD8 T cell        15725
mature NK                 6074
memory B cell             4106
memory CD4 T cell        22197
mid erythroid             2192
monocyte-DC                515
myelocyte                 6675
myeloid DC progenitor      110
naive B cell             19265
naive CD4 T cell          5873
naive CD8 T cell          8

In [37]:
from scipy import sparse
array_vals = adata.X
adata.X = sparse.csr_matrix(adata.X)
adata.raw = adata
adata.X = array_vals

In [38]:
adata = adata[adata.obs['cell.labels'].isin(["CD14 monocyte", "DC precursor", "DC1", "DC2", "DC3", 
                     "promonocyte", 
                     "myelocyte", "myeloid DC progenitor", "MOP", "neutrophil", "pDC", "pDC progenitor", 
                     "promyelocyte", "tDC", "CD16 monocyte", "monocyte-DC"])].copy()

In [39]:
cell_numbers = adata.obs.groupby(["cell.labels"]).apply(len)
cell_numbers

cell.labels
CD14 monocyte            3670
CD16 monocyte            1938
DC1                       135
DC2                       481
DC3                       550
DC precursor              462
MOP                      1440
monocyte-DC               515
myelocyte                6675
myeloid DC progenitor     110
neutrophil               2482
pDC                      1134
pDC progenitor             63
promonocyte              7448
promyelocyte             2197
tDC                        75
dtype: int64

In [40]:
adata.obs["dataset"] = "ABM"

In [41]:
cell_numbers = adata.obs.groupby(["sample", "cell.labels"]).apply(len)
pd.DataFrame(cell_numbers).to_csv("/Users/b8058304/Documents/PhD_work/Coding/manuscript_figs/resources_for_pipelines/abm_myeloid_nrs_by_sample_20200724.csv")
cell_numbers

sample     cell.labels   
MantonBM1  CD14 monocyte      755
           CD16 monocyte      244
           DC1                 55
           DC2                 83
           DC3                 95
                             ... 
MantonBM6  pDC                305
           pDC progenitor      21
           promonocyte       2018
           promyelocyte       544
           tDC                 18
Length: 64, dtype: int64

# Import the CB data

In [42]:
adata = sc.read('/Users/b8058304/Documents/PhD_work/Coding/cord_blood/data/cord_blood_raw_dr_20200717.h5ad')

In [43]:
cb = adata

In [44]:
adata

AnnData object with n_obs × n_vars = 148442 × 33694 
    obs: 'sample', 'lanes', 'mad_prd', 'auto_prd', 'cell.labels', 'cell.labels2', 'leiden', 'percent.mito', 'nGene', 'nUMI', 'cell.labels_20200717'
    var: 'gene_ids'
    obsm: 'X_orig_pca', 'X_pca', 'X_umap'

In [45]:
cell_numbers = adata.obs.groupby(["cell.labels"]).apply(len)
cell_numbers

cell.labels
CD8 T cell          16345
CD14 monocyte       13324
CD16 monocyte         888
CD56 bright NK       4066
CMP                   272
DC1                    67
DC2                   155
DC precursor          169
GMP                   203
HSC                   194
ILC precursor        1519
MEMP                  338
MK                   1262
early MK              496
early erythroid       532
late erythroid        878
mature NK            7860
mid erythroid        2627
myelocyte            3726
naive B cell        19516
naive CD4 T cell    69338
neutrophil           3458
pDC                   242
preDC                 269
promonocyte           607
tDC                    91
dtype: int64

In [46]:
adata.obs["dataset"] = "CB"

# Run numbers per sample cross-tissue 

In [47]:
cell_numbers = ys.obs.groupby(["orig.ident"]).apply(len)
pd.DataFrame(cell_numbers).to_csv("/Users/b8058304/Documents/PhD_work/Coding/manuscript_figs/resources_for_pipelines/ys_by_sample_20200724.csv")
cell_numbers

orig.ident
F32    1181
F35     448
F37    8442
dtype: int64

In [48]:
cell_numbers = liver.obs.groupby(["orig.ident"]).apply(len)
pd.DataFrame(cell_numbers).to_csv("/Users/b8058304/Documents/PhD_work/Coding/manuscript_figs/resources_for_pipelines/liver_by_sample_20200724.csv")
cell_numbers

orig.ident
F16     1135
F17     1992
F21     6657
F22     9958
F23     5658
F29     5315
F30     9730
F32     6753
F33    16945
F34    15401
F35     8821
F38     9425
F41     6904
F45     8369
dtype: int64

In [49]:
cell_numbers =fbm.obs.groupby(["orig.ident"]).apply(len)
pd.DataFrame(cell_numbers).to_csv("/Users/b8058304/Documents/PhD_work/Coding/manuscript_figs/resources_for_pipelines/fbm_by_sample_20200724.csv")
cell_numbers

orig.ident
F21         3142
F29         8162
F30        10414
F38         3115
F41         7910
F45         6605
F50        15899
F51        28997
SB19PCW    20318
dtype: int64

In [50]:
cell_numbers = abm.obs.groupby(["sample"]).apply(len)
pd.DataFrame(cell_numbers).to_csv("/Users/b8058304/Documents/PhD_work/Coding/manuscript_figs/resources_for_pipelines/abm_by_sample_20200724.csv")
cell_numbers

sample
MantonBM1    34634
MantonBM2    38256
MantonBM5    36678
MantonBM6    32458
dtype: int64

In [51]:
cell_numbers = cb.obs.groupby(["sample"]).apply(len)
pd.DataFrame(cell_numbers).to_csv("/Users/b8058304/Documents/PhD_work/Coding/manuscript_figs/resources_for_pipelines/cb_by_sample_20200724.csv")
cell_numbers

sample
MantonCB1    24437
MantonCB2    40383
MantonCB5    40750
MantonCB6    42872
dtype: int64

# Run numbers for all cells per sample cross tissue

In [52]:
cell_numbers = ys.obs.groupby(["orig.ident", "cell.labels"]).apply(len)
pd.DataFrame(cell_numbers).to_csv("/Users/b8058304/Documents/PhD_work/Coding/manuscript_figs/resources_for_pipelines/ys_by_sample_allcells_20200724.csv")
cell_numbers

orig.ident  cell.labels            
F32         DC progenitor                 1
            Early mast cell               9
            Erythroid AB                792
            Erythroid EZ                 10
            Erythroid M                 261
            ILC precursor                 9
            Lymphoid progenitor          12
            MEMP                         16
            Monocyte                      2
            Myeloid progenitor            3
            NK early                      8
            yolk sac Macrophage I        22
            yolk sac Macrophage II       28
            yolk sac progenitor/MPP       8
F35         DC progenitor                13
            Early mast cell              37
            Endothelium                  23
            Erythroid EZ                 22
            Erythroid M                  11
            ILC precursor                15
            Lymphoid progenitor          14
            MEMP                        

In [53]:
cell_numbers = ys.obs.groupby(["orig.ident", "cell.labels_progen"]).apply(len)
pd.DataFrame(cell_numbers).to_csv("/Users/b8058304/Documents/PhD_work/Coding/manuscript_figs/resources_for_pipelines/ys_by_sample_allcells_progenreclus_20200724.csv")
cell_numbers

orig.ident  cell.labels_progen
F32         CMP                     3
            ELP                    12
            GMP                     1
            HSC                     5
            MEMP                    8
            MEP                     8
            macrophage              2
F35         CMP                    11
            ELP                    19
            GMP                     2
            HSC                    19
            MEMP                   23
            MEP                    38
            Monocyte precursor      2
            macrophage              1
            neut myeloid            6
F37         CMP                    41
            ELP                   143
            GMP                    37
            HSC                    81
            MEMP                   24
            MEP                    10
            Monocyte precursor     24
            macrophage             39
            neut myeloid           12
dtype: int64

In [54]:
cell_numbers = liver.obs.groupby(["orig.ident", "cell.labels"]).apply(len)
pd.DataFrame(cell_numbers).to_csv("/Users/b8058304/Documents/PhD_work/Coding/manuscript_figs/resources_for_pipelines/liver_by_sample_allcells_20200724.csv")
cell_numbers

orig.ident  cell.labels         
F16         B cell                    3
            DC1                       3
            DC2                      10
            DC precursor              1
            Early Erythroid         196
                                   ... 
F45         Pre pro B cell           58
            VCAM1+ EI macrophage     68
            pDC precursor            17
            pre-B cell              337
            pro-B cell              323
Length: 372, dtype: int64

In [55]:
cell_numbers = liver.obs.groupby(["orig.ident", "cell.labels_progen"]).apply(len)
pd.DataFrame(cell_numbers).to_csv("/Users/b8058304/Documents/PhD_work/Coding/manuscript_figs/resources_for_pipelines/liver_by_sample_allcells_progenreclus_20200724.csv")
cell_numbers

orig.ident  cell.labels_progen    
F16         ELP                        4
            GMP                        3
            HSC                        1
            MEMP                       4
            MEP                        9
                                      ..
F45         eo/baso/mast precursor     8
            erythroid-MPP hybrid       1
            macrophage                18
            myeloid DC progenitor     11
            pDC progenitor             4
Length: 169, dtype: int64

In [56]:
cell_numbers =fbm.obs.groupby(["orig.ident", "cell.labels"]).apply(len)
pd.DataFrame(cell_numbers).to_csv("/Users/b8058304/Documents/PhD_work/Coding/manuscript_figs/resources_for_pipelines/fbm_by_sample_allcells_20200724.csv")
cell_numbers

orig.ident  cell.labels       
F21         CD4 T cell             12
            CD8 T cell             13
            CD14 monocyte         217
            CD56 bright NK          6
            CMP                    19
                                 ... 
SB19PCW     promyelocyte          667
            schwann cells           8
            sinusoidal EC          42
            stromal macrophage     40
            tip EC                105
Length: 529, dtype: int64

In [57]:
cell_numbers = abm.obs.groupby(["sample", "cell.labels"]).apply(len)
pd.DataFrame(cell_numbers).to_csv("/Users/b8058304/Documents/PhD_work/Coding/manuscript_figs/resources_for_pipelines/abm_by_sample_allcells_20200724.csv")
cell_numbers

sample     cell.labels        
MantonBM1  CD14 monocyte          755
           CD16 monocyte          244
           CD56 bright NK         240
           CLP                    195
           CMP                     79
                                 ... 
MantonBM6  promyelocyte           544
           stroma                  37
           tDC                     18
           transitional B cell    204
           transitional NK        130
Length: 176, dtype: int64

In [58]:
cell_numbers = cb.obs.groupby(["sample", "cell.labels"]).apply(len)
pd.DataFrame(cell_numbers).to_csv("/Users/b8058304/Documents/PhD_work/Coding/manuscript_figs/resources_for_pipelines/cb_by_sample_allcells_20200724.csv")
cell_numbers

sample     cell.labels   
MantonCB1  CD8 T cell        1739
           CD14 monocyte     5633
           CD16 monocyte      242
           CD56 bright NK     543
           CMP                 50
                             ... 
MantonCB6  neutrophil        1719
           pDC                 74
           preDC              146
           promonocyte        219
           tDC                 63
Length: 104, dtype: int64

# Rerun after neut myeloid/monocyte precursor reclustering of FL

In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
import scipy.stats
import anndata
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.axes._axes import _log as matplotlib_axes_logger
from scipy import sparse
matplotlib_axes_logger.setLevel('ERROR')
# silence NumbaPerformanceWarning
import warnings
from numba.errors import NumbaPerformanceWarning
warnings.filterwarnings("ignore", category=NumbaPerformanceWarning)

/Users/b8058304/.pyenv/versions/3.6.4/envs/py36/lib/python3.6/site-packages/anndata/_core/anndata.py:21: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex


In [2]:
sc.settings.verbosity = 1  # verbosity: errors (0), warnings (1), info (2), hints (3)
# Set up the plot config for viewing the annotation clearly.
sc.settings.set_figure_params(dpi=120, dpi_save=1000)
sc.logging.print_versions()

scanpy==1.4.4 anndata==0.7.1 umap==0.3.10 numpy==1.17.1 scipy==1.4.1 pandas==1.0.5 scikit-learn==0.22.2.post1 statsmodels==0.11.1 python-igraph==0.8.0 louvain==0.6.1


In [4]:
adata=sc.read("/Users/b8058304/Documents/PhD_work/Coding/manuscript_figs/data/fig2_fl_myeloid_reannotation_20200722.h5ad")

In [5]:
cell_numbers = adata.obs.groupby(["orig.ident", "cell.labels"]).apply(len)
pd.DataFrame(cell_numbers).to_csv("/Users/b8058304/Documents/PhD_work/Coding/manuscript_figs/resources_for_pipelines/liver_by_sample_neut_myeloid_mono_pre_reclus_20200727.csv")
cell_numbers

orig.ident  cell.labels 
F16         MOP              5
            promonocyte      1
F17         MOP             11
            monocyte         3
            promonocyte      8
            promyelocyte    11
F21         MOP             20
            monocyte         8
            promonocyte      6
            promyelocyte     9
F22         MOP             29
            monocyte        14
            promonocyte      4
            promyelocyte    22
F23         MOP             14
            monocyte        20
            promonocyte      5
            promyelocyte    11
F29         MOP             12
            monocyte         3
            promonocyte      2
            promyelocyte    11
F30         MOP             16
            monocyte        34
            promonocyte     12
            promyelocyte    13
F32         MOP             15
            monocyte        30
            promonocyte     11
            promyelocyte     9
F33         MOP             46
            mo